# Библиотеки и гиперпараметры

In [47]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchdiffeq
import numpy as np
import matplotlib.pyplot as plt
import scipy
from tqdm import tqdm
from scipy.optimize import minimize
from torch.utils.data import DataLoader, TensorDataset
import os
import random
from torchsummary import summary

#from scipy.ndimage import minimum_filter


# Параметры
is_restart = True
p_cutoff = 0.01
n_epoch = 600
n_plot = 50
datasize = 100
tstep = 0.4
n_exp_train = 20
n_exp_test = 10
n_exp = n_exp_train + n_exp_test
noise = 5.e-2
ns = 5
nr = 4
k = torch.tensor([0.1, 0.2, 0.13, 0.3], dtype=torch.float32)  
atol = 1e-5
rtol = 1e-2
maxiters = 10000
lb = 1e-5
ub = 10.0
b0 = -10.0

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



# Система ОДУ

In [50]:
# Функция ODE

def true_ode_func(t, y, k):
    dydt = torch.zeros_like(y)
    dydt[0] = -2 * k[0] * y[0]**2 - k[1] * y[0]
    dydt[1] = k[0] * y[0]**2 - k[3] * y[1] * y[3]    
    dydt[2] = k[1] * y[0] - k[2] * y[2]
    dydt[3] = k[2] * y[2] - k[3] * y[1] * y[3]
    dydt[4] = k[3] * y[1] * y[3]
    return dydt



# Генерация начальных и истинных данных

In [60]:

# Генерация данных

np.random.seed()
torch.manual_seed(123)
u0_list = np.random.rand(n_exp, ns).astype(np.float32)
u0_list[:, :2] += 0.2
u0_list[:, 2:] = 0.0    # u0_list - матрица (30x5) нач.значений концентраций для 30 экспериментов в диапазоне: [0.2 0.2 0 0 0] - [1.2 1.2 0 0 0]

#print("u0_list: ", u0_list) 
tsteps = torch.linspace(0, datasize * tstep, datasize, dtype=torch.float32) #временной промежуток от 0 до 40 по 100 точек через 0.4
#print(tsteps)
ode_data_list = []
print(ode_data_list)



std_list = []

for i in range(n_exp):   #   Вычисление точных изменений концентраций со временем
    u0 = torch.tensor(u0_list[i], dtype=torch.float32)
    #print("u0 torch tensor ", i, ": ",u0)
    
    # Решение системы ОДУ с использованием torchdiffeq
    sol = torchdiffeq.odeint(lambda t, y: true_ode_func(t, y, k), u0, tsteps, atol=atol, rtol=rtol, method='dopri5')              
    #print(sol.shape)
    
    # Добавление шума к данным
    noise_tensor = torch.randn_like(sol) * sol * noise
    ode_data = sol + noise_tensor
    
    # Сохранение данных
    ode_data_list.append(ode_data)
    
    # Вычисление стандартного отклонения
    std_list.append(torch.max(ode_data, dim=0).values - torch.min(ode_data, dim=0).values + lb)
    
ode_data_list = torch.stack(ode_data_list)  #(30, 100, 5) изменения концентраций 5 веществ на 100 временных точках для каждого из 30 экспериментов

##########
#TESTING
##########

#print(ode_data_list[1])

# testing = ode_data_list[1].numpy()
# np.set_printoptions(
#     precision=8,   # Количество знаков после запятой
#     suppress=True  # Отключаем научную нотацию
# )
# print(testing)


# Объединение результатов и вычисление максимальных значений
y_std = torch.max(torch.stack(std_list), dim=0).values
print(y_std)



[]
[[0.6477024  0.6663213  0.         0.         0.        ]
 [0.53905576 0.6363582  0.04887135 0.00131286 0.00003164]
 [0.4924546  0.6579696  0.08161089 0.00447291 0.00023686]
 [0.42812642 0.75425446 0.11724032 0.00942825 0.00081319]
 [0.34495747 0.60800856 0.14025429 0.01648129 0.00176569]
 [0.33616337 0.6205885  0.1661435  0.02179183 0.00327284]
 [0.30486578 0.673631   0.18991706 0.02940558 0.00525903]
 [0.26826504 0.6765972  0.20549001 0.0372499  0.00832882]
 [0.23901688 0.624502   0.19892934 0.04354737 0.01258754]
 [0.21742132 0.66921675 0.24557891 0.04916299 0.01528479]
 [0.2258176  0.5783192  0.23116198 0.05539289 0.01947566]
 [0.17291857 0.6357755  0.23464859 0.07249875 0.02174402]
 [0.1600125  0.69585127 0.23355326 0.0772907  0.03042921]
 [0.14259334 0.62553406 0.2329655  0.07971769 0.03355669]
 [0.14303415 0.58834577 0.23472099 0.08561447 0.04053808]
 [0.13357304 0.65016735 0.23757356 0.08973932 0.05170467]
 [0.12983917 0.68206763 0.22609332 0.09875005 0.05599599]
 [0.1074124

# CRNN Model

In [63]:
# Определение модели CRNN
class CRNN(nn.Module):
    def __init__(self):
        super(CRNN, self).__init__()
        self.w_b = nn.Parameter(torch.randn(nr) + b0)
        w_out_clamped  = torch.clamp(torch.randn(ns, nr),-2.5, 2.5)
        w_out_clamped[torch.abs(w_out_clamped) < p_cutoff] = 0.0
        self.w_out = nn.Parameter(w_out_clamped)

        print("w_in: ",  torch.clamp(-self.w_out, 0, 2.5).T)
    
    def forward(self, t, u):
        
        w_in = torch.clamp(-self.w_out, 0, 2.5)
        
        w_in_x = torch.matmul(w_in.T, torch.log(torch.clamp(u, lb, ub))) 
        du = torch.matmul(self.w_out, torch.exp(w_in_x + self.w_b))

        return du

    # def printInfo(self):

    #     print("Bias w_b: \n     ", self.w_b )
    #     print("Output w_out: \n     ", self.w_out)

    #     print("Grad of Bias : \n     ",  self.w_b.grad)
    #     print("Grad of Output : \n     ",  self.w_out.grad)
        
##########
#TESTING
##########

# model = CRNN()
# print(model.w_out.data)
# print(f"Model structure: {model}\n")
# gen_p = model.parameters() # возвращает генератор с набором параметров
# print(list(model.parameters())) # отображение списка параметров

#model.printInfo()


w_in:  tensor([[0.6600, 0.0000, 0.0000, 0.4709, 1.4033],
        [0.0000, 1.9188, 0.0000, 0.0000, 0.6057],
        [0.8968, 1.4377, 0.0000, 0.1935, 0.0000],
        [1.7302, 2.5000, 1.5946, 1.1369, 0.0000]], grad_fn=<PermuteBackward0>)
tensor([[-0.6600,  0.4327, -0.8968, -1.7302],
        [ 0.1499, -1.9188, -1.4377, -2.5000],
        [ 0.0712,  0.1655,  0.0290, -1.5946],
        [-0.4709,  0.6804, -0.1935, -1.1369],
        [-1.4033, -0.6057,  0.5546,  0.3441]])
Model structure: CRNN()

[Parameter containing:
tensor([-10.0202,  -9.1111,  -9.6397, -10.8175], requires_grad=True), Parameter containing:
tensor([[-0.6600,  0.4327, -0.8968, -1.7302],
        [ 0.1499, -1.9188, -1.4377, -2.5000],
        [ 0.0712,  0.1655,  0.0290, -1.5946],
        [-0.4709,  0.6804, -0.1935, -1.1369],
        [-1.4033, -0.6057,  0.5546,  0.3441]], requires_grad=True)]


# Предсказание

In [65]:
def predict_neuralode(Model, u0):


    solution = torchdiffeq.odeint(func=Model, y0=u0, t=tsteps, method='dopri5', atol=1e-5, rtol=1e-2, options={'max_num_steps': 10000})
    solution = torch.clamp(solution, -10.0, 10.0)
    return solution

    
##########
#TESTING
##########

# model = CRNN()
# print(model.w_b.data.clone())
# print(model.w_out.data.clone())
# w_b_test = model.w_b.data.clone()
# w_out_test = model.w_out.data.clone()

# u0 = torch.tensor(u0_list[1], dtype=torch.float32)
# print("u0", u0)
# y = model(0.0, u0)
# pred=predict_neuralode(model, u0)
# print("y: ", y)
# print(pred)
# mae = torch.mean(torch.abs(pred - ode_data_list[1]))
# print(mae)

w_in:  tensor([[0.2779, 0.2422, 0.0000, 0.6124, 1.0513],
        [0.0000, 0.6172, 0.0000, 0.0000, 1.5539],
        [1.6458, 0.0000, 0.6416, 0.0000, 0.0000],
        [1.5123, 0.0000, 1.1132, 0.0000, 0.0000]], grad_fn=<PermuteBackward0>)
tensor([-10.6305, -11.4045, -10.0712,  -8.9618])
tensor([[-0.2779,  1.2409, -1.6458, -1.5123],
        [-0.2422, -0.6172,  0.3169,  0.4598],
        [ 0.5910,  0.2146, -0.6416, -1.1132],
        [-0.6124,  1.9505,  0.3941,  0.4645],
        [-1.0513, -1.5539,  0.4710,  1.7062]])
u0 tensor([0.6403, 0.6161, 0.0000, 0.0000, 0.0000])
y:  tensor([-2.0969e-08,  4.0670e-09, -8.2669e-09,  5.0390e-09,  6.2262e-09],
       grad_fn=<MvBackward0>)
tensor([[ 6.4026e-01,  6.1608e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 6.4026e-01,  6.1608e-01, -3.3402e-09,  2.0360e-09,  2.5156e-09],
        [ 6.4026e-01,  6.1608e-01, -6.6803e-09,  4.0719e-09,  5.0312e-09],
        [ 6.4026e-01,  6.1608e-01, -1.0021e-08,  6.1079e-09,  7.5469e-09],
        [ 6.4026e-01,  6

# Функция потерь

In [67]:
def loss_function(predict, target):
    mae = torch.mean(torch.abs(predict - target)/y_std)
    return mae



# Отображение параметров

In [70]:
def display_params(w_b, w_out):
    
    print("species (column) reaction (row)")
    w_in = torch.clamp(-w_out, 0, 2.5)
    
    print("\nw_in")
    print(torch.round(w_in.mT * 1000) / 1000)  # Округление до 3 знаков через масштабирование
    
    print("\nw_b")
    exp_w_b = torch.exp(w_b)
    print(torch.round(exp_w_b * 1000) / 1000)
    
    print("\nw_out")
    print(torch.round(w_out.mT * 1000) / 1000)
    
    print("\n\n####_display_params worked_####")

display_params(w_b_test, w_out_test)

species (column) reaction (row)

w_in
tensor([[0.2780, 0.2420, 0.0000, 0.6120, 1.0510],
        [0.0000, 0.6170, 0.0000, 0.0000, 1.5540],
        [1.6460, 0.0000, 0.6420, 0.0000, 0.0000],
        [1.5120, 0.0000, 1.1130, 0.0000, 0.0000]])

w_b
tensor([0., 0., 0., 0.])

w_out
tensor([[-0.2780, -0.2420,  0.5910, -0.6120, -1.0510],
        [ 1.2410, -0.6170,  0.2150,  1.9500, -1.5540],
        [-1.6460,  0.3170, -0.6420,  0.3940,  0.4710],
        [-1.5120,  0.4600, -1.1130,  0.4650,  1.7060]])


####_display_params worked_####


# Графики концентраций

In [73]:
def cbi(predict, i_exp):

    os.makedirs('figs_experiments_batches', exist_ok=True)
    species = ["A", "B", "C", "D", "E"]
    ode_data = ode_data_list[i_exp].T  # (ns, datasize)
    #print(ode_data.T)
    predicted = predict.T.detach().numpy()
    #print(predicted.shape)
    
    fig, axes = plt.subplots(nrows=ns, figsize=(6, 12), sharex=True)
    for i in range(ns):
        ax = axes[i]
        ax.scatter(tsteps.numpy(), ode_data[i].numpy(), label="Exp", facecolors='none', edgecolors='black')
        ax.plot(tsteps.numpy(), predicted[i], label="CRNN", color='orange')
        ax.set_ylabel(f"[{species[i]}]")
        if i == 0:
            ax.legend(frameon=False)
    
    axes[-1].set_xlabel("Time")
    plt.tight_layout()
    plt.savefig(f"figs_experiments_batches/i_exp_{i_exp}.png")
    plt.close()
    #plt.show()
    return False

# Вызов функции для i-го эксперимента
cbi(pred, 1)

False

# Графики функции потерь

In [76]:
list_loss_train = []
list_loss_val = []
iter_count = 1
def cb(Model, loss_train, loss_val):
    global list_loss_train, list_loss_val, iter_count


    list_loss_train.append(loss_train)
    list_loss_val.append(loss_val)

    w_b_disp = Model.w_b.data.clone()
    w_out_disp = Model.w_out.data.clone()

    if iter_count % n_plot == 0:
        display_params(w_b_disp, w_out_disp)

        print(f"min loss train: {min(list_loss_train):.4e}, val: {min(list_loss_val):.4e}")

        list_exp = [random.randint(0, n_exp - 1)]
        print("list_exp: ",list_exp)
        print(f"update plot for {list_exp}")

        for i_exp in list_exp:
            print("\n updated plot_",i_exp)
            u0 = torch.tensor(u0_list[i_exp], dtype=torch.float32)
            predict = predict_neuralode(Model, u0)
            cbi(predict, i_exp)

        # График лосса
        plt.figure()
        plt.plot(list_loss_train, label="train", color='blue')
        plt.plot(list_loss_val, label="val", color='orange')
        #plt.xscale("log")
        #plt.yscale("log")
        plt.xlabel("Iteration")
        plt.ylabel("Loss")
        plt.legend()
        plt.title("Training & Validation Loss")
        plt.grid(True)
        plt.savefig(f"figs_experiments_batches/loss_plot.png")
        #plt.show()
        plt.close()

    iter_count += 1

# Разбиение на батчи

In [79]:
# Преобразование списков в тензоры
u0_tensor = torch.tensor(u0_list[:n_exp_train], dtype=torch.float32)  # только обучающие эксперименты
ode_data_tensor = ode_data_list[:n_exp_train]  # (n_exp_train, datasize, ns)

# Создание TensorDataset и DataLoader
train_dataset = TensorDataset(u0_tensor, ode_data_tensor)
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

# Проверка данных в DataLoader
for i, (batch_u0, batch_target) in enumerate(train_loader):
    print(f"Batch {i + 1}:")
    print("Batch u0 (initial conditions):")
    print(batch_u0)  # Начальные условия для батча
    print("Batch target (ODE solutions):")
    print(batch_target.shape)  # Целевые данные (решения ОДУ)
    print("-" * 50)

Batch 1:
Batch u0 (initial conditions):
tensor([[0.2312, 0.3353, 0.0000, 0.0000, 0.0000],
        [0.8863, 0.7576, 0.0000, 0.0000, 0.0000],
        [0.8055, 1.1339, 0.0000, 0.0000, 0.0000],
        [0.4253, 0.9999, 0.0000, 0.0000, 0.0000],
        [1.1749, 1.1415, 0.0000, 0.0000, 0.0000],
        [0.8586, 0.3949, 0.0000, 0.0000, 0.0000],
        [1.0171, 0.4393, 0.0000, 0.0000, 0.0000],
        [0.5021, 0.5886, 0.0000, 0.0000, 0.0000],
        [0.8208, 0.7832, 0.0000, 0.0000, 0.0000],
        [0.9793, 0.6931, 0.0000, 0.0000, 0.0000]])
Batch target (ODE solutions):
torch.Size([10, 100, 5])
--------------------------------------------------
Batch 2:
Batch u0 (initial conditions):
tensor([[0.8062, 0.2038, 0.0000, 0.0000, 0.0000],
        [0.6403, 0.6161, 0.0000, 0.0000, 0.0000],
        [0.7266, 0.6308, 0.0000, 0.0000, 0.0000],
        [0.6613, 0.7795, 0.0000, 0.0000, 0.0000],
        [0.9293, 0.6589, 0.0000, 0.0000, 0.0000],
        [0.4165, 0.3530, 0.0000, 0.0000, 0.0000],
        [0.73

# Обучение модели

In [177]:
model = CRNN()#.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01)
loss_epoch = torch.zeros(n_exp, dtype=torch.float32)  # для логов
epochs = tqdm(range(n_epoch))

for epoch in epochs:
    for u0_batch, target_batch in train_loader:
        optimizer.zero_grad()

        # Получение батчевого предсказания (batch_size, datasize, ns)
        pred_batch = []
        for u0 in u0_batch:
            pred = predict_neuralode(model, u0)
            pred_batch.append(pred.unsqueeze(0))  # добавляем batch dim

        pred_batch = torch.cat(pred_batch, dim=0)

        # Вычисление loss
        loss = torch.mean(torch.abs(pred_batch - target_batch) / y_std)

        loss.backward()
        optimizer.step()

        model.w_out.data.clamp_(-2.5, 2.5)
        model.w_out.data[torch.abs(model.w_out.data) < p_cutoff] = 0.0

    ######### Validation ##########

    with torch.no_grad():  # блок без градиентов
        for i_exp in range(n_exp):
            u0 = torch.tensor(u0_list[i_exp], dtype=torch.float32)
            target = ode_data_list[i_exp]

            pred = predict_neuralode(model, u0)  # forward
            loss_epoch[i_exp] = torch.mean(torch.abs(pred - target) / y_std)

    loss_train = torch.mean(loss_epoch[:n_exp_train]).item()
    loss_val = torch.mean(loss_epoch[n_exp_train:]).item()

    # Вывод в tqdm
    epochs.set_description(f"Loss train {loss_train:.4e} val {loss_val:.4e}")

    # Сохранение логов или callback
    cb(model, loss_train, loss_val)


w_in:  tensor([[0.0417, 0.0000, 0.1511, 0.0924, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.6895, 0.0000],
        [0.0000, 0.8158, 0.0625, 0.0000, 0.0000],
        [0.0000, 0.2403, 0.0000, 0.0000, 0.4110]], grad_fn=<PermuteBackward0>)


Loss train 2.7802e-01 val 2.5215e-01:   8%|▊         | 49/600 [01:05<11:55,  1.30s/it]

species (column) reaction (row)

w_in
tensor([[0.4300, 0.0000, 0.4440, 0.3950, 0.0000],
        [0.0420, -0.0000, 0.0000, 0.0000, 0.0000],
        [-0.0000, -0.0000, 0.0000, 0.0000, 0.0000],
        [0.1730, -0.0000, 0.0000, 0.0000, 0.0000]])

w_b
tensor([0.0000, 0.0000, 0.0010, 0.0000])

w_out
tensor([[-0.4300,  0.4740, -0.4440, -0.3950,  1.1520],
        [-0.0420,  0.0000,  2.0330,  0.2780,  2.0000],
        [ 0.0000,  0.0000,  0.2820,  2.4520,  1.5400],
        [-0.1730,  0.0000,  1.1640,  1.6910,  0.3770]])


####_display_params worked_####
min loss train: 2.7802e-01, val: 2.5215e-01
list_exp:  [16]
update plot for [16]

 updated plot_ 16


Loss train 2.3400e-01 val 2.1555e-01:  16%|█▋        | 99/600 [02:11<10:57,  1.31s/it]

species (column) reaction (row)

w_in
tensor([[-0.0000, 0.0470, 0.0000, -0.0000, 0.0000],
        [-0.0000, 0.0770, 0.0000, 0.0000, 0.0000],
        [0.8300, -0.0000, 0.0000, 0.0000, 0.0000],
        [1.4330, -0.0000, 0.0000, 0.0000, 0.0000]])

w_b
tensor([0.0000, 0.0010, 0.0040, 0.0010])

w_out
tensor([[ 0.0000, -0.0470,  0.0960,  0.0000,  2.5000],
        [ 0.0000, -0.0770,  0.5970,  0.0360,  2.5000],
        [-0.8300,  0.0000,  0.1160,  1.8960,  2.5000],
        [-1.4330,  0.0000,  0.8310,  1.4350,  1.1880]])


####_display_params worked_####
min loss train: 2.3400e-01, val: 2.1555e-01
list_exp:  [5]
update plot for [5]

 updated plot_ 5


...

Loss train 2.4409e-02 val 2.5312e-02:  92%|█████████▏| 549/600 [16:26<01:47,  2.11s/it]

species (column) reaction (row)

w_in
tensor([[0.0260, 0.8400, 0.0000, 0.7630, 0.0000],
        [-0.0000, 0.1370, 0.8860, 0.0640, 0.0000],
        [0.3440, 0.0000, 0.0000, 0.0000, 0.3540],
        [1.2260, 0.0000, 0.0000, 0.0000, -0.0000]])

w_b
tensor([0.1730, 0.1910, 0.0620, 0.2490])

w_out
tensor([[-0.0260, -0.8400,  0.3450, -0.7630,  0.4700],
        [ 0.0000, -0.1370, -0.8860, -0.0640,  0.3820],
        [-0.3440,  0.0130,  0.3060,  1.9940, -0.3540],
        [-1.2260,  0.2480,  0.8640,  0.0680,  0.0000]])


####_display_params worked_####
min loss train: 2.4409e-02, val: 2.4656e-02
list_exp:  [12]
update plot for [12]

 updated plot_ 12


Loss train 2.3116e-02 val 2.3062e-02: 100%|█████████▉| 599/600 [18:16<00:02,  2.22s/it]

species (column) reaction (row)

w_in
tensor([[0.0230, 0.8460, 0.0000, 0.8130, 0.0000],
        [0.0000, 0.1010, 0.9310, 0.0600, 0.0000],
        [0.3750, -0.0000, 0.0000, 0.0000, 0.3440],
        [1.2240, 0.0000, 0.0000, 0.0000, -0.0000]])

w_b
tensor([0.2070, 0.1870, 0.0670, 0.2450])

w_out
tensor([[-0.0230, -0.8460,  0.1990, -0.8130,  0.5220],
        [ 0.0450, -0.1010, -0.9310, -0.0600,  0.3630],
        [-0.3750,  0.0000,  0.3780,  1.9920, -0.3440],
        [-1.2240,  0.2320,  0.8680,  0.0530,  0.0000]])


####_display_params worked_####
min loss train: 2.3116e-02, val: 2.2927e-02
list_exp:  [19]
update plot for [19]

 updated plot_ 19


Loss train 2.3116e-02 val 2.3062e-02: 100%|██████████| 600/600 [18:16<00:00,  1.83s/it]
